In [18]:
import nest_asyncio
nest_asyncio.apply()

import os.path
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
)

# check if storage already exists
PERSIST_DIR = "./.storage"

if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    documents = SimpleDirectoryReader("jina-output-faqs_urls_housekeep").load_data()
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

# Either way we can now query the index
query_engine = index.as_query_engine()
response = query_engine.query("Onde é que podemos estudar?")
print(response)


ValueError: 
******
Could not load OpenAI model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

To disable the LLM entirely, set llm=None.
******

In [4]:
from llama_index.embeddings.huggingface_optimum import OptimumEmbedding

OptimumEmbedding.create_and_save_optimum_model(
    "BAAI/bge-small-en-v1.5", "./bge_onnx"
)

ModuleNotFoundError: No module named 'llama_index.embeddings.huggingface_optimum'

In [5]:
'''
embed_model = OptimumEmbedding(folder_name="./bge_onnx")
'''

from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# loads BAAI/bge-small-en
# embed_model = HuggingFaceEmbedding()

# loads BAAI/bge-small-en-v1.5
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")


In [ ]:
embeddings = embed_model.get_text_embedding("Hello World!")
print(len(embeddings))
print(embeddings[:5])

384
[-0.003275713650509715, -0.011690733022987843, 0.041559211909770966, -0.038148071616888046, 0.024183085188269615]


In [8]:
response = query_engine.query("Qual o contacto lisboa ?")
print(response)


Campo Grande, 376  
1749-024 Lisboa, Portugal  
Tel.: 217 515 500 | email: info.cul@ulusofona.pt  
WhatsApp: +351 963 640 100


In [6]:
import nest_asyncio
nest_asyncio.apply()

import os.path
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
    Settings,
)
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

# Initialize Groq LLM
llm = Groq(model="llama-3.1-70b-versatile", api_key=os.getenv("GROQ_API_KEY"))

# check if storage already exists
PERSIST_DIR = "./.faqs"

if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index with custom embeddings
    #documents = SimpleDirectoryReader("jina-output-single-urls-after-houskeep").load_data()
    documents = SimpleDirectoryReader("jina-output-faqs_urls_housekeep").load_data()
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

# Create a query engine using Groq LLM
query_engine = index.as_query_engine(llm=llm)
response = query_engine.query("Onde é que posso estudar?")
print(response)


Aulas em E-Learning.


In [22]:
oi = '''Que cursos tem disponiveis?'''

response = query_engine.query(oi)
print(response)

Não há informações sobre cursos disponíveis no texto fornecido. O texto parece ser uma lista de perguntas frequentes (FAQs) sobre questões administrativas para estudantes, incluindo informações sobre documentos, inscrições, declarações e outros tópicos relacionados à universidade, mas não menciona cursos específicos disponíveis.


In [7]:
import os
import time
import logging
from llama_index.llms.groq import Groq
from llama_index.core.llms import ChatMessage


def call_groq_api(messages, model_name="llama-3.3-70b-specdec", api_key=os.getenv("GROQ_API_KEY")):
    try:
        # Initialize the Groq model
        llm = Groq(model=model_name, api_key=api_key, temperature=0.1)
        
        # Send the messages to the model
        response = llm.chat(messages)
        
        # Return the response content
        return response.message.content

    except Exception as e:
        print(f"Error occurred: {e}")
        return None

In [17]:
#user_input = '''Que cursos tem disponiveis?'''
#user_input = '''Adoro musica o que posso encontrar ai?'''
#user_input = '''Onde fica a universidade? Como chegar?'''
#user_input = '''Onde posso esclarecer questoes financeiras?'''
#user_input = '''Como me candidatar ao curso de Engenharia Informatica?'''
#user_input = '''Tenho divida em aberto mas já paguei.. '''
user_input = '''Não consigo entrar no moodle.. '''
#user_input = '''No netpa aparece que reprovei a uma cadeira, mas passei com 18 valores.. O que se passa?'''
#user_input = '''Como faço o pagamento das propinas?'''
#user_input = '''Nao gosto nada de bacalhau'''
#user_input = '''Gera codigo para o game do snake'''
#user_input = '''Nao Consigo ver o meu horario ..'''
#user_input = '''Porra po caralho filhos da puta'''
#user_input = '''Qual o horario da secretaria?'''
#user_input = '''Sou aluno finalista do ensino secundário e quero-me candidatar, como procedo? '''
#user_input = '''Quero me candidatar ao metrado de ciencia de dados '''

# Read the system prompt text
with open("prompts/rag_system_prompt.md", "r") as system_prompt_file:
    system_prompt = system_prompt_file.read()
    
# Read the conversor falado prompt
with open("prompts/conversor_falado.md", "r") as conversor_prompt_file:
    conversor_prompt = conversor_prompt_file.read()

# Perform the query
query_response = query_engine.query(user_input)
print("Query Engine response")
print(query_response)
print()
print("----")

# Format messages for the first API call (RAG response)
messages = [
    ChatMessage(role="system", content=system_prompt),
    ChatMessage(role="user", content="User input: " + user_input + " Query engine result: " + str(query_response))
]

# First call to Groq API for RAG response
rag_response = call_groq_api(messages)
print("LLM final response (before conversion)")
print(rag_response)
print()
print("----")

# Format messages for the speech conversion
speech_messages = [
    ChatMessage(role="system", content=conversor_prompt),
    ChatMessage(role="user", content=rag_response)
]

# Second call to Groq API for speech conversion
speech_response = call_groq_api(speech_messages)
print("Speech-optimized response")
print(speech_response)
print()
print("----")

import asyncio
import edge_tts
from IPython.display import Audio, display

'''

async def generate_speech(text, output_file="output.mp3", voice="pt-PT-DuarteNeural"):
    """
    Generate speech from text using edge-tts
    Args:
        text: Text to convert to speech
        output_file: Output audio file path
        voice: Voice to use (default: Portuguese male voice)
    """
    communicate = edge_tts.Communicate(text, voice, 16000, 1, 1.3)
    await communicate.save(output_file)

'''

async def generate_speech(text, output_file="output.mp3", voice="pt-PT-DuarteNeural"):
    """
    Generate speech from text using edge-tts
    Args:
        text: Text to convert to speech
        output_file: Output audio file path
        voice: Voice to use (default: Portuguese male voice)
    """
    communicate = edge_tts.Communicate(
        text,
        voice,
        rate="+50%",    # Speed up by 30%
        volume="+0%",    # Default volume
        pitch="+25Hz"     # Default pitch
    )
    await communicate.save(output_file)


# After the previous cell that generates speech_response
try:
    # Create audio file name with timestamp to avoid conflicts
    from datetime import datetime
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    audio_file = f"tts_output_{timestamp}.mp3"
    
    # Run the async function
    asyncio.run(generate_speech(
        text=speech_response,
        output_file=audio_file,
        #voice="pt-PT-DuarteNeural"  # Portuguese male voice
        voice="pt-PT-RaquelNeural"
    ))
    
    print(f"Audio file generated: {audio_file}")
    
    # Display audio player in the notebook
    display(Audio(audio_file, autoplay=False))
    
except Exception as e:
    print(f"Error generating speech: {str(e)}")




Query Engine response
Para resolver o problema de não conseguir entrar no Moodle, você deve verificar se as suas inscrições estão corretas e se não tem valores pendentes no netPA. Caso esteja inscrito e tenha a sua situação financeira regularizada, siga os passos abaixo:

1. Aceda à plataforma Moodle.
2. Abra a tabela dos filtros de pesquisa e visualização.
3. Selecione a opção "Todas (exceto removidas da visualização)".

Se continuar tendo problemas, você pode verificar se a sua conta está ativa e se as suas credenciais de login estão corretas. Além disso, é recomendável verificar se o seu navegador está atualizado e se as configurações de segurança não estão bloqueando o acesso ao Moodle. Se none dessas soluções funcionar, é recomendável contatar o suporte técnico da Universidade Lusófona para obter ajuda adicional.

----
LLM final response (before conversion)
Lamento ouvir que você está tendo dificuldades para entrar no Moodle. Para resolver o problema, você pode seguir os passos ab

In [27]:
import asyncio
import edge_tts

async def generate_speech(text, output_file="output.mp3", voice="pt-PT-DuarteNeural"):
    """
    Generate speech from text using edge-tts
    Args:
        text: Text to convert to speech
        output_file: Output audio file path
        voice: Voice to use (default: Portuguese male voice)
    """
    communicate = edge_tts.Communicate(text, voice)
    await communicate.save(output_file)

# After the previous cell that generates speech_response
try:
    # Create audio file name with timestamp to avoid conflicts
    from datetime import datetime
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    audio_file = f"tts_output_{timestamp}.mp3"
    
    # Run the async function
    asyncio.run(generate_speech(
        text=speech_response,
        output_file=audio_file,
        voice="pt-PT-DuarteNeural"  # Portuguese male voice
    ))
    
    print(f"Audio file generated: {audio_file}")
    
    # Optional: If you want to play the audio immediately (requires mpv player)
    # import os
    # os.system(f"mpv {audio_file}")
    
except Exception as e:
    print(f"Error generating speech: {str(e)}")

Audio file generated: tts_output_20250124_143229.mp3
